In [ ]:
# Setting my folder directory
# Spacings are fine!

from os import chdir
path = "C://Users//nelso//Documents//Data Science//Python//Scripts//pySpark"
chdir(path)

In [ ]:
import findspark
findspark.init("C:\devtools\spark-2.4.4-bin-hadoop2.7")
print("Hello! findspark ran fine.")


In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

sc = SparkContext("local", "first app")

# from pyspark.sql import SparkSession

print("SparkContext initialized.")

SparkContext initialized.


In [9]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Testing PySpark dataframes") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

print('SparkSession Initialized')
df = spark.read.load("/home/cloudera/Desktop/RI.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

df.select("Recipe").show(5)

SparkSession Initialized
+--------------------+
|              Recipe|
+--------------------+
|Basil, Roasted Pe...|
|Crispy Cheese Twists|
|   Mom's Yeast Rolls|
|Sweet Potato Bread I|
|         Orange Buns|
+--------------------+
only showing top 5 rows



In [10]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Testing PySpark dataframes") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

print('SparkSession Initialized')
df1 = spark.read.load("hdfs://quickstart.cloudera:8020/user/cloudera/RI.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

df1.select("Recipe").show(5)



SparkSession Initialized
+--------------------+
|              Recipe|
+--------------------+
|Basil, Roasted Pe...|
|Crispy Cheese Twists|
|   Mom's Yeast Rolls|
|Sweet Potato Bread I|
|         Orange Buns|
+--------------------+
only showing top 5 rows



In [14]:
sc.addPyFile("/home/cloudera/Desktop/pytest.py")

In [11]:
#####Using inbuilt tokenizers

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

tokenizer = Tokenizer(inputCol="Recipe", outputCol="words")
#pipeline = Pipeline(stages=[tokenizer])

wordsData = tokenizer.transform(df1.select("Recipe"))
wordsData.show(5)

+--------------------+--------------------+
|              Recipe|               words|
+--------------------+--------------------+
|Basil, Roasted Pe...|[basil,, roasted,...|
|Crispy Cheese Twists|[crispy, cheese, ...|
|   Mom's Yeast Rolls|[mom's, yeast, ro...|
|Sweet Potato Bread I|[sweet, potato, b...|
|         Orange Buns|      [orange, buns]|
+--------------------+--------------------+
only showing top 5 rows



In [13]:
#####Using user defined functions
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import *

mywords = ['basil','yeast','buns']

def lowercase(x):
    lowerW = x.lower()
    return lowerW

def matching(x):
    return bool(set(x) & set(mywords))

def find_letters(x):
    first_cap = re.search('[a-z]',x).group(0)
    return first_cap

lowercasing_udf = udf(lowercase, StringType())
matching_udf = udf(matching, BooleanType())
letters_udf = udf(find_letters, StringType())

wordsData2 = df1.withColumn('lowercase', lowercasing_udf('Recipe')).select('Recipe','lowercase')
wordsData2.show(5)

wordsData3 = wordsData.withColumn('match',matching_udf('words')).select('Recipe','words','match') \
.withColumn('letters', letters_udf('Recipe')).select('Recipe','words','match','letters')
wordsData3.show(5)

+--------------------+--------------------+
|              Recipe|           lowercase|
+--------------------+--------------------+
|Basil, Roasted Pe...|basil, roasted pe...|
|Crispy Cheese Twists|crispy cheese twists|
|   Mom's Yeast Rolls|   mom's yeast rolls|
|Sweet Potato Bread I|sweet potato bread i|
|         Orange Buns|         orange buns|
+--------------------+--------------------+
only showing top 5 rows

+--------------------+--------------------+-----+-------+
|              Recipe|               words|match|letters|
+--------------------+--------------------+-----+-------+
|Basil, Roasted Pe...|[basil,, roasted,...|false|      a|
|Crispy Cheese Twists|[crispy, cheese, ...|false|      r|
|   Mom's Yeast Rolls|[mom's, yeast, ro...| true|      o|
|Sweet Potato Bread I|[sweet, potato, b...|false|      w|
|         Orange Buns|      [orange, buns]| true|      r|
+--------------------+--------------------+-----+-------+
only showing top 5 rows



In [17]:
#####Using user defined functions
from pyspark.sql.functions import udf
from pyspark.sql.types import *

mywords = ['basil','yeast','buns']

def find_letters(x):
    return pytest.myfx(x)

letters_udf = udf(find_letters, StringType())

wordsData3 = wordsData.withColumn('letters', letters_udf('Recipe')).select('Recipe','words','letters')
wordsData3.show(5)

In [8]:
#####Using user defined functions - from NLTK package 
### THIS SECTION DOESNT WORK
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import re

# word tokenizer
# This will not work, because each slave node does not have the NLTK installed!
import nltk
nltk.data.path.append("C://Users//nelso//Documents//Data Science//Python")
def nltk_tok(x):
    token = nltk.word_tokenize(x)
    return token
print(nltk_tok("Hello!, Big data."))

token_udf = udf(nltk_tok, StringType())

wordsData4 = df.withColumn('tokens', token_udf('Recipe')).select('Recipe','tokens')
wordsData4.show(5)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/cloudera/nltk_data'
    - '/opt/rh/rh-python36/root/usr/nltk_data'
    - '/opt/rh/rh-python36/root/usr/share/nltk_data'
    - '/opt/rh/rh-python36/root/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - 'C://Users//nelso//Documents//Data Science//Python'
    - 'C://Users//nelso//Documents//Data Science//Python'
    - ''
**********************************************************************


In [16]:
from pytest import myfx
myfx("hellfdasfadsfads")

'h'

In [ ]:
with open('mytest.txt') as file:
    myfile = file.read().splitlines()
    
print(myfile)

In [ ]:
from pyspark.sql.types import Row

mydata = sc.textFile('sampledf.csv')

a = mydata \
.map(lambda line: line.split(',')) \
.collect()

print(a)

#df = mydata.toDF()

#df.show(5)

In [ ]:
data = [1,2,3,4,5,6,8,9,10]

# sc = pyspark.SparkContext()
a = sc.parallelize(data).map(lambda x: x*x).collect()
print (a)